# Necessary Packages

In [2]:
############################################################
# Importing the Necessary Packages
############################################################
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import Planck15 as cosmo
import astropy.units as u
from astropy.constants import c, G
import pygravlens as gl

# Mass Density to Convergence

In [3]:
def mtok(density_map, z_lens, z_source = 2.0):
    dL = cosmo.angular_diameter_distance(z_lens)
    dS = cosmo.angular_diameter_distance(z_source)
    dLS = cosmo.angular_diameter_distance_z1z2(z_lens,z_source)
    #compute critical density and convert to Msun per kpc^2
    crit = ((c)**2 /(4*np.pi*G) *dS/(dL*dLS)).to(u.Msun/(u.kpc)**2)
    #and convert to Msun per arcsec^2
    dist = dL.to(u.kiloparsec)
    crit = (crit * dist**2 /u.rad**2).to(u.Msun/u.arcsec**2)
    print(f'Critcal density: {crit:.2e}')
    
    kappa = density_map/crit.value
    return kappa

# Redshift Arrays

In [6]:
# First, I need a dictionary to link a snapshot number to its redshift. I could have used the header if only I had saved it
reds = [1.00, 0.95, 0.92, 0.89, 0.85, 0.82, 0.79, 0.76, 0.73, 0.70, 0.68, 0.64, 0.62, 0.60, 0.58, 0.55, 0.52, 0.50, 0.48, 0.46, 0.44, 0.42, 0.40, 0.38, 0.36, 0.35, 0.33, 0.31, 0.30, 0.27, 0.26, 0.24, 0.23, 0.21, 0.20, 0.18, 0.17,0.15, 0.14, 0.13, 0.11, 0.10, 0.08, 0.07, 0.06, 0.05, 0.03, 0.02, 0.01, 0.0]
sr = {}
for i in np.arange(50, 100, 1):
    sr[i] = reds[i - 50]

# Converter

In [28]:
def dosnap(snap):
    dens_maps = {}
    def_maps = {}
    
    for i in range(len(snap)):
        angle_per_ckpc = cosmo.arcsec_per_kpc_comoving(sr[snap[i]]).value
        print(f'Snapshot {snap[i]}, z = {sr[snap[i]]}, angular scale = {angle_per_ckpc:.3f} arcsec/ckpc or {1/angle_per_ckpc:.2f} ckpc/arcsec')


        # box size, in arcsec
        Lbox_arcsec = 75000/cosmo.h * angle_per_ckpc

        # density maps - dm
        dens_map_dm = np.load('density_'+str(snap[i])+'_dm.npy', allow_pickle = 'TRUE')
        dens_map_dm /= angle_per_ckpc**2
        # stars
        dens_map_stars = np.load('density_'+str(snap[i])+'_stars.npy', allow_pickle = 'TRUE')
        dens_map_stars /=angle_per_ckpc**2
        # gas
        dens_map_gas = np.load('density_'+str(snap[i])+'_gas.npy', allow_pickle = 'TRUE')
        dens_map_gas /=angle_per_ckpc**2
        # total
        global comb_dens_map
        comb_dens_map = dens_map_dm + dens_map_stars + dens_map_gas
        
        dens_maps[snap[i]] = comb_dens_map

        # number of pixels
        npix = comb_dens_map.shape[0]
        # pixel scale
        pixel_scale = Lbox_arcsec/npix
        print(f'Pixel scale = {pixel_scale:.2f} arcsec/pix')
        # x and y arrays (1d)
        global x1d, y1d
        x1d = np.arange(-Lbox_arcsec/2,Lbox_arcsec/2,pixel_scale)
        y1d = np.arange(-Lbox_arcsec/2,Lbox_arcsec/2,pixel_scale)
        # shift by half a pixel so points give pixel *centers*
        x1d += 0.5*pixel_scale
        y1d += 0.5*pixel_scale

        # calculate the kappa map
        kappa_comb = mtok(comb_dens_map, sr[snap[i]])
        # difference relative to mean
        kappa_mean = np.mean(kappa_comb)
        dkappa = kappa_comb - kappa_mean
        print('Mean kappa =',kappa_mean)

        # calculate lensing quantities
        phi,phix,phiy,phixx,phiyy,phixy = gl.kappa2lens(x1d,y1d,dkappa)
        defnorm = np.sqrt(phix**2+phiy**2)
        
        def_maps[snap[i]] = defnorm

    np.save(f'dens_maps_dictionary_{snap}.npy', dens_maps)
    np.save(f'def_maps_dictionary_{snap}.npy', def_maps)